In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import time 
import collections

from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    confusion_matrix,
    classification_report
)

from tensorflow.python.platform import build_info as tf_build_info
from collections import Counter
print(tf_build_info.build_info)

In [ ]:
data = pd.read_csv("C:/models/data/truss_rebalanced.csv", quotechar='"')

In [ ]:
X = data.drop(["material_cost_per_truss", "build_cost_per_truss", "setup_cost_per_run"], axis=1)
y = data[["material_cost_per_truss", "build_cost_per_truss", "setup_cost_per_run"]]
X = X.astype(float)
y = y.astype(float)

print(X.dtypes)
print(y.dtypes)
print(X.isna().sum())

In [ ]:
# ==================================================
# BUILD COST CLASSIFICATION MODEL DATA SETUP
# ==================================================

# --- 1. Prepare target and encode labels ---
y_build = y[["build_cost_per_truss"]].copy()  # Single target column
enc_build = LabelEncoder()
y_build_labels = enc_build.fit_transform(y_build.values.ravel())  # Convert to class IDs (0..N-1)

# --- Merge rare classes with nearest neighbor ---
counts = Counter(y_build_labels)
class_info = pd.DataFrame({
    "class_id": np.arange(len(enc_build.classes_)),
    "build_cost": enc_build.classes_.astype(float),
    "count": [counts.get(i, 0) for i in range(len(enc_build.classes_))]
})

threshold = 100  # minimum samples per class
y_merged = y_build_labels.copy()

# Sort by build cost for easier neighbor lookup
class_info_sorted = class_info.sort_values("build_cost").reset_index(drop=True)

for _, row in class_info_sorted.iterrows():
    cls = row["class_id"]
    count = row["count"]

    if count < threshold:
        cost = row["build_cost"]

        # Find valid classes (≥ threshold)
        valid_classes = class_info_sorted[class_info_sorted["count"] >= threshold]

        # Find the nearest valid class by cost difference
        nearest_idx = (valid_classes["build_cost"] - cost).abs().idxmin()
        nearest_class = valid_classes.loc[nearest_idx, "class_id"]
        nearest_cost = valid_classes.loc[nearest_idx, "build_cost"]

        # Merge (relabel) all occurrences
        y_merged[y_merged == cls] = nearest_class

        print(f"Class {int(cls):2d} (${cost:.2f}, {count} samples) → merged into "
              f"Class {int(nearest_class):2d} (${nearest_cost:.2f})")

new_counts = Counter(y_merged)

df_counts = pd.DataFrame({
    "Class_ID": list(new_counts.keys()),
    "Build_Cost_$": [float(enc_build.classes_[int(cls)]) for cls in new_counts.keys()],
    "Samples": list(new_counts.values())
}).sort_values("Build_Cost_$").reset_index(drop=True)

pd.set_option("display.max_rows", None)  # show all rows
print(df_counts)
    
y_build_labels = y_merged

num_classes_build = len(enc_build.classes_)
build_cost_levels = enc_build.classes_.astype(float)  # Actual $ values per class

# --- 2. Split (stratified) ---
x_train_build, x_test_build, y_train_build_lbl, y_test_build_lbl = train_test_split(
    X, y_build_labels,
    test_size=0.2,
    random_state=42,
)

# --- 3. Scale features ---
x_scaler_build = StandardScaler()
x_train_build_scaled = x_scaler_build.fit_transform(x_train_build)
x_test_build_scaled  = x_scaler_build.transform(x_test_build)

# --- 4. Compute class weights (optional but helpful) ---
# All possible classes (model expects 0 to num_classes-1)
all_classes = np.arange(num_classes_build)

# Classes actually present in training data
present_classes = np.unique(y_train_build_lbl)

# Compute weights only for present classes
weights_arr = compute_class_weight(
    class_weight='balanced',
    classes=present_classes,
    y=y_train_build_lbl
)

# Map weights to present classes
present_weights = dict(zip(present_classes, weights_arr))

# Build full class_weight dict for model
class_weights_build = {
    int(cls): float(present_weights.get(cls, 1.0))  # default to 1.0 if class not in training
    for cls in all_classes
}

In [ ]:
# --- 5. Build classification model ---
model_build_cls = models.Sequential([
    layers.Input(shape=(x_train_build_scaled.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes_build, activation='softmax')  # Classification head
])

model_build_cls.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='top3_acc')
    ]
)

# --- 6. Train ---
print("Training Build Cost CLASSIFIER...")

reduce_lr_build = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5
)
early_stop_build = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
with tf.device('/GPU:0'):
    history_build_cls = model_build_cls.fit(
        x_train_build_scaled, y_train_build_lbl,
        validation_split=0.2,
        epochs=100,
        batch_size=256,
        verbose=1,
        callbacks=[reduce_lr_build, early_stop_build], 
    )

In [8]:

# Tensor of actual price values (replace with your enc_build.classes_ array)
build_class_prices = tf.constant(enc_build.classes_.astype(np.float32))

# Input for post-processing wrapper
inp = tf.keras.Input(shape=(x_train_build_scaled.shape[1],))
probs = model_build_cls(inp)
class_idx = tf.argmax(probs, axis=1, output_type=tf.int32)
pred_price = tf.gather(build_class_prices, class_idx)

model_build_price = tf.keras.Model(inputs=inp, outputs=pred_price, name="build_cost_price_model")
model_build_price.save("C:/models/build_cost_price_model")
print("✅ Build cost model now outputs actual $ value per sample.")


INFO:tensorflow:Assets written to: C:/models/build_cost_price_model\assets
✅ Build cost model now outputs actual $ value per sample.


In [ ]:
# ==================================================
# EVALUATE BUILD COST CLASSIFIER (CLEAN REPORT)
# ==================================================

from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

# --- Make predictions on test set ---
probs_test_build = model_build_cls.predict(x_test_build_scaled)
pred_build_lbl = probs_test_build.argmax(axis=1)
pred_build_argmax = build_cost_levels[pred_build_lbl]

# Optional: compute "expected value" for smoother continuous predictions
pred_build_ev = probs_test_build @ build_cost_levels
y_true_build = build_cost_levels[y_test_build_lbl]

# ==================================================
# VISUALIZE TRAINING HISTORY (BUILD COST MODEL)
# ==================================================
plt.figure(figsize=(8,5))
plt.plot(history_build_cls.history['loss'], label='Training Loss')
plt.plot(history_build_cls.history['val_loss'], label='Validation Loss')
plt.title('Build Cost Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# ==================================================
# VISUAL COMPARISONS
# ==================================================

plt.figure(figsize=(7, 6))
plt.scatter(y_true_build, pred_build_argmax, alpha=0.7, edgecolor='k')
plt.plot([y_true_build.min(), y_true_build.max()],
         [y_true_build.min(), y_true_build.max()],
         'r--', lw=2)
plt.title("Actual vs Predicted Build Cost per Truss (Argmax Prediction)")
plt.xlabel("Actual Build Cost ($)")
plt.ylabel("Predicted Build Cost ($)")
plt.grid(True)
plt.show()

plt.figure(figsize=(7, 6))
plt.scatter(y_true_build, pred_build_ev, alpha=0.7, color='orange', edgecolor='k')
plt.plot([y_true_build.min(), y_true_build.max()],
         [y_true_build.min(), y_true_build.max()],
         'r--', lw=2)
plt.title("Actual vs Predicted Build Cost per Truss (Expected Value Prediction)")
plt.xlabel("Actual Build Cost ($)")
plt.ylabel("Predicted Build Cost ($)")
plt.grid(True)
plt.show()

# ==================================================
# COMPUTE METRICS
# ==================================================
report_lines = []
report_lines.append("=== BUILD COST CLASSIFIER EVALUATION REPORT ===\n")

# --- Confusion Matrix ---
cm_build = confusion_matrix(y_test_build_lbl, pred_build_lbl)
report_lines.append("📊 Confusion Matrix (numeric class IDs):\n")
report_lines.append(pd.DataFrame(cm_build).to_string(index=False))
report_lines.append("\n")

# --- Classification Report ---
report_lines.append("📋 Classification Report (Precision, Recall, F1-Score):\n")
class_report = classification_report(y_test_build_lbl, pred_build_lbl, digits=3)
report_lines.append(class_report)
report_lines.append("\n")

# --- Overall Accuracy ---
correct_preds = np.trace(cm_build)
total_preds = np.sum(cm_build)
accuracy = correct_preds / total_preds
report_lines.append("✅ OVERALL ACCURACY:\n")
report_lines.append(f"The model correctly classified {accuracy*100:.2f}% of build-cost samples.\n\n")

# --- Per-Class Accuracy ---
class_totals = cm_build.sum(axis=1)
correct_per_class = np.diag(cm_build)
per_class_accuracy = correct_per_class / class_totals
per_class_table = pd.DataFrame({
    "Class ID": np.arange(len(per_class_accuracy)),
    "Accuracy (%)": (per_class_accuracy * 100).round(2)
})
report_lines.append("🎯 PER-CLASS ACCURACY (% of correctly predicted samples per cost bin):\n")
report_lines.append(per_class_table.to_string(index=False))
report_lines.append("\n")

# --- Identify worst misclassification ---
errors = pred_build_lbl != y_test_build_lbl
wrong_idxs = np.where(errors)[0]
if len(wrong_idxs) > 0:
    confidences = probs_test_build[np.arange(len(probs_test_build)), y_test_build_lbl]
    worst_idx = wrong_idxs[np.argmin(confidences[wrong_idxs])]

    worst_true = y_test_build_lbl[worst_idx]
    worst_pred = pred_build_lbl[worst_idx]
    worst_prob = probs_test_build[worst_idx][pred_build_lbl[worst_idx]]
    true_conf = confidences[worst_idx]

    report_lines.append("😬 WORST CLASSIFICATION (lowest confidence on true class):\n")
    report_lines.append(f"True Class: {worst_true} | Predicted Class: {worst_pred}\n")
    report_lines.append(f"Predicted Probability: {worst_prob:.2f} | Confidence on True Class: {true_conf:.2f}\n\n")
else:
    report_lines.append("✅ No misclassifications detected!\n\n")

# ==================================================
# DOLLAR-LEVEL ERROR METRICS
# ==================================================
y_true_dollars = build_cost_levels[y_test_build_lbl]
y_pred_dollars = build_cost_levels[pred_build_lbl]

mae = mean_absolute_error(y_true_dollars, y_pred_dollars)
rmse = np.sqrt(mean_squared_error(y_true_dollars, y_pred_dollars))
dollar_errors = np.abs(y_true_dollars - y_pred_dollars)
max_error = dollar_errors.max()
large_miss_idx = np.where(dollar_errors > 20)[0]

report_lines.append("💰 DOLLAR-LEVEL PERFORMANCE METRICS:\n")
report_lines.append(f"- Mean Absolute Error (MAE): ${mae:.2f}\n")
report_lines.append("  → Average difference between predicted and actual build cost.\n")
report_lines.append(f"- Root Mean Square Error (RMSE): ${rmse:.2f}\n")
report_lines.append("  → Typical magnitude of prediction error, weighted toward larger misses.\n")
report_lines.append(f"- Maximum Absolute Error: ${max_error:.2f}\n")
report_lines.append("  → The single largest dollar deviation observed.\n")
report_lines.append(f"- Samples with >$20 error: {len(large_miss_idx)} out of {len(dollar_errors)} total.\n")
report_lines.append("  → Extreme outliers beyond a $20 difference.\n\n")

# ==================================================
# CLASS DISTRIBUTION
# ==================================================
counts = Counter(y_train_build_lbl)
plt.figure(figsize=(8, 5))
plt.bar(counts.keys(), counts.values(), color="slateblue")
plt.title("Training Class Distribution (Build Cost)")
plt.xlabel("Class ID")
plt.ylabel("Samples per Class")
plt.grid(axis="y", alpha=0.3)
plt.show()


# Optional: summary printout
print("\n=== Summary Metrics ===")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Square Error (RMSE): ${rmse:.2f}")
print(f"Maximum Absolute Error: ${max_error:.2f}")
print(f"Samples >$20 error: {len(large_miss_idx)} / {len(dollar_errors)}")

# ==================================================
# PRINT FEATURES OF LARGE-ERROR PREDICTIONS
# ==================================================
error_threshold = 10
large_error_idx = np.where(dollar_errors > error_threshold)[0]

if len(large_error_idx) > 0:
    print(f"\n=== 🔍 Predictions with Error > ${error_threshold} ===")
    
    # Convert scaled test set to DataFrame for readability
    feature_names = getattr(x_test_build_scaled, 'columns', None)
    if feature_names is None:
        # if it's a NumPy array, make generic feature names
        feature_names = [f"feature_{i}" for i in range(x_test_build_scaled.shape[1])]
    
    df_large_errors = pd.DataFrame(
        x_test_build_scaled[large_error_idx],
        columns=feature_names
    )
    df_large_errors["Actual ($)"] = y_true_dollars[large_error_idx]
    df_large_errors["Predicted ($)"] = y_pred_dollars[large_error_idx]
    df_large_errors["Absolute Error ($)"] = dollar_errors[large_error_idx]

    # Sort by absolute error descending
    df_large_errors = df_large_errors.sort_values("Absolute Error ($)", ascending=False)

    # Print top few rows and save to CSV for review
    print(df_large_errors.head(20).to_string(index=False))
    df_large_errors.to_csv(f"large_error_predictions_{timestamp}.csv", index=False)
    print(f"\n💾 Saved all {len(df_large_errors)} large-error samples to 'large_error_predictions_{timestamp}.csv'")
else:
    print(f"\n✅ No samples found with error greater than ${error_threshold}.")
    
    
# First, attach class IDs (based on nearest build_cost_levels)
def nearest_class_id(values, class_levels):
    return [np.argmin(np.abs(class_levels - v)) for v in values]

df_large_errors["Actual Class"] = nearest_class_id(df_large_errors["Actual ($)"], build_cost_levels)
df_large_errors["Pred Class"]   = nearest_class_id(df_large_errors["Predicted ($)"], build_cost_levels)

# Optional: readable labels
df_large_errors["Actual Label"] = [build_cost_levels[i] for i in df_large_errors["Actual Class"]]
df_large_errors["Pred Label"]   = [build_cost_levels[i] for i in df_large_errors["Pred Class"]]

# Pick the span-related feature (looks like feature_6 drives most outliers)
span_feature = "feature_6"

plt.figure(figsize=(8,6))
sc = plt.scatter(
    df_large_errors["Actual Class"],
    df_large_errors["Pred Class"],
    c=df_large_errors[span_feature],
    cmap="viridis",
    s=80,
    alpha=0.8,
    edgecolor="k"
)
plt.colorbar(sc, label=f"{span_feature} (scaled)")
plt.title("High-Error Samples: Actual vs. Predicted Class")
plt.xlabel("Actual Class ID")
plt.ylabel("Predicted Class ID")
plt.grid(True, alpha=0.4)
plt.show()



In [ ]:
import os
import joblib
import tensorflow as tf

def export_build_cost_classifier(model, x_scaler, label_encoder, class_weights, export_dir="build_cost_classifier"):
    """
    Exports a trained build cost classification model, scaler, label encoder, and metadata.
    """
    os.makedirs(export_dir, exist_ok=True)

    # --- Save model ---
    model.save(os.path.join(export_dir, "model_build_classifier.keras"))

    # --- Save preprocessing tools ---
    joblib.dump(x_scaler, os.path.join(export_dir, "x_scaler_build.pkl"))
    joblib.dump(label_encoder, os.path.join(export_dir, "label_encoder_build.pkl"))
    joblib.dump(class_weights, os.path.join(export_dir, "class_weights_build.pkl"))

    # --- Metadata ---
    metadata = {
        "num_classes": len(label_encoder.classes_),
        "class_labels": label_encoder.classes_.astype(float).tolist()
    }
    joblib.dump(metadata, os.path.join(export_dir, "metadata_build.pkl"))

    print(f"✅ Build Cost Classifier exported to '{export_dir}'")


In [ ]:
export_build_cost_classifier(
    model_build_cls,
    x_scaler_build,
    enc_build,
    class_weights_build,
    export_dir="exported_build_classifier"
)